## **EDA Project**


**Data cleaning**

En este notebook se realizará la limpieza y tratamiento de los datos extraídos con anterioridad


In [75]:
# Importamos las librerías necesarias
import os
import pandas as pd
import numpy as np
import sys
from importlib import reload
reload(Main)
sys.path.append('../scripts')
from Main import check_nans, desglose_nans, drop_nans, interpolar_nans, add_poblacion



El primer paso será unir todos los csv donde tenemos los datos de turistas por país con temporalidad mensual en un mismo

In [7]:
# Carpeta raíz donde están las subcarpetas con los CSV
carpeta_raiz = '../data/Tourism_jpgs'

# DataFrame vacío para ir agregando todos los datos
df_final = pd.DataFrame(columns=['Year', 'Month', 'Country', 'N_visitors'])


In [8]:

# Recorrer subcarpetas
for subcarpeta in os.listdir(carpeta_raiz):
    ruta_sub = os.path.join(carpeta_raiz, subcarpeta)
    
    # Verificar que sea una carpeta
    if os.path.isdir(ruta_sub):
        # Buscar CSV en la carpeta
        for archivo in os.listdir(ruta_sub):
            if archivo.endswith('.csv'):
                ruta_csv = os.path.join(ruta_sub, archivo)
                
                # Leer CSV
                df_crudo = pd.read_csv(ruta_csv)
                
                # Obtener los nombres de las columnas de países (todas menos Year y Month)
                paises = [col for col in df_crudo.columns if col not in ['Year', 'Month']]
                
                # Convertir a formato largo, manteniendo NaNs
                datos_largos = []
                for _, fila in df_crudo.iterrows():
                    año = fila['Year']
                    mes = fila['Month']
                    for pais in paises:
                        visitantes = fila[pais]
                        datos_largos.append([año, mes, pais, visitantes])
                
                df_largo = pd.DataFrame(datos_largos, columns=['Year', 'Month', 'Country', 'N_visitors'])
                
                # Agregar al DataFrame final
                df_final = pd.concat([df_final, df_largo], ignore_index=True)

# Mostrar resultado
print(df_final)


         Year  Month      Country N_visitors
0      2010.0    1.0   Unnamed: 0        0.0
1      2010.0    1.0      Albania        NaN
2      2010.0    1.0        Egypt   998440.0
3      2010.0    1.0      Morocco        NaN
4      2010.0    1.0  Switzerland   535966.0
...       ...    ...          ...        ...
10546  2024.0    4.0       Turkey  3582567.0
10547  2024.0    5.0   Unnamed: 0        4.0
10548  2024.0    5.0     Thailand  2633464.0
10549  2024.0    5.0      Tunisia        NaN
10550  2024.0    5.0       Turkey  4958978.0

[10551 rows x 4 columns]


C:\Users\David\AppData\Local\Temp\ipykernel_15848\3504646436.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, df_largo], ignore_index=True)


In [10]:
df_final.to_csv("../data/tourism.csv", index=False) # Guardamos el DataFrame final en un CSV para poder trabajar con el mas adelante

### **CARGA DEL DATASET**

In [12]:
df = pd.read_csv("../data/tourism.csv")
df.sample(23)   

,Year,Month,Country,N_visitors
8901,2022.0,8.0,Mexico,3093307.0
3944,2018.0,1.0,Philippines,732506.0
7344,2022.0,6.0,Malaysia,971574.0
2910,2015.0,4.0,Unnamed: 0,3.0
470,2017.0,11.0,Unnamed: 0,10.0
1125,2014.0,5.0,Unnamed: 0,4
2954,2015.0,12.0,Uzbekistan,NaN
9133,2012.0,2.0,Portugal,324004.0
7132,2018.0,12.0,Hungary,3367000.0
4523,2013.0,4.0,Japan,923017.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10551 entries, 0 to 10550
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Year        10551 non-null  float64
 1   Month       10551 non-null  float64
 2   Country     10551 non-null  object 
 3   N_visitors  10014 non-null  object 
dtypes: float64(2), object(2)
memory usage: 329.8+ KB


### **EXPLORACION DEL DATASET & LIMPIEZA Y NORMALIZACION**

Primero cambiaremos el tipo de cada columna

In [14]:
# A continuación, cambiamos el tipo de cada columna
df['Year'] = pd.to_datetime(df['Year'], format='%Y').dt.year
df['Month'] = pd.to_datetime(df['Month'], format='%m').dt.month
df['Country'] = df['Country'].astype(str)


Al intentar convertir a float N_visitors da error debido a que se han generado datos con barra en vez del simbolo menos (-) debido al scrapeo.

Se realiza un replace para solventarlo.

In [17]:
df['N_visitors'] = df['N_visitors'].astype(str).str.replace('—', '-', regex=False).str.strip() # Replace de -
df['N_visitors'] = pd.to_numeric(df['N_visitors'], errors='coerce') # Conversion a float 
df['N_visitors'] = df['N_visitors'].round(0).astype('Int64') # Conversion a int
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10551 entries, 0 to 10550
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year        10551 non-null  int32 
 1   Month       10551 non-null  int32 
 2   Country     10551 non-null  object
 3   N_visitors  10014 non-null  Int64 
dtypes: Int64(1), int32(2), object(1)
memory usage: 257.7+ KB


In [18]:
# Una vez hemos modificado los tipos de cada columna, hacemos un check de los valores negativos que vimos anteriormente
df_negativos = df.loc[df['N_visitors'] < 0, ['Country', 'Year', 'Month', 'N_visitors']]
print(df_negativos)


        Country  Year  Month  N_visitors
1437  Australia  2019      7     -790380


In [20]:
# Este error es debido al scrapeo a traves de imagenes, debemos corregirlo y cambiarlo a positivo
df.loc[(df['Country'] == 'Australia') & (df['Year'] == 2019) & (df['Month'] == 7), 'N_visitors'] = 790380
print(df.loc[(df['Country'] == 'Australia') & (df['Year'] == 2019) & (df['Month'] == 7)])

      Year  Month    Country  N_visitors
1437  2019      7  Australia      790380


A continuación se realizará un tratamiento de los NaNs.

Para ello realizaremos un check para afinar el numero de NaNs por Year y Country, contabilizando el número de estos para tener una visión más clara.

In [42]:
check_nans(df)

Total de NaNs: 537
Total desglosado:
Country
Hungary                     117
Uzbekistan                  101
United Arab Emirates         64
Poland                       61
Albania                      56
Colombia                     38
Vietnam                      36
Morocco                      13
Bulgaria                     12
Switzerland                   8
United Kingdom                5
Egypt                         5
Malaysia                      4
Italy                         3
United States of America      2
Denmark                       2
Argentina                     1
Philippines                   1
Indonesia                     1
South Korea                   1
Sweden                        1
India                         1
Tunisia                       1
Belgium                       1
Canada                        1
Australia                     1
Name: N_visitors, dtype: int64


Como se puede observar hay una gran cantidad de NaNs.

Para tener una visión mas completa de las causas se ejecutará un def el cual:
         Devuelve un diccionario con los años donde cada país tiene NaN en N_visitors y la cantidad de NaNs por año.


    Parámetros:
    df           : DataFrame con columnas 'Country', 'Year', 'N_visitors'
    lista_paises : lista de países a consultar
    
    Retorna:
    diccionario {pais: {'años_nan': [...], 'nans_por_año': {año: n_NaNs}}}


En primer lugar se realizara un check de aquellos paises con mayor peso: 'Hungary', 'Uzbekistan', 'United Arab Emirates', 'Poland', 'Albania', 'Colombia', 'Vietnam', 'Morocco', 'Bulgaria', 'Switzerland'

In [ ]:

# Lista de países que quieres analizar
paises = [
    'Hungary', 'Uzbekistan', 'United Arab Emirates', 'Poland',
    'Albania', 'Colombia', 'Vietnam', 'Morocco', 'Bulgaria', 'Switzerland'
]

# Llamar a la función
resumen_años_nan = desglose_nans(df_final, paises)

# Mostrar resultado
for pais, info in resumen_años_nan.items():
    print(pais)
    print("Años con NaN:", info['años_nan'])
    print("Número de NaNs por año:", info['nans_por_año'])
    print()


Hungary
Años con NaN: [2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0, 2022.0, 2023.0, 2024.0]
Número de NaNs por año: {2010.0: 8, 2011.0: 8, 2012.0: 8, 2013.0: 8, 2014.0: 8, 2015.0: 8, 2016.0: 8, 2017.0: 8, 2018.0: 8, 2019.0: 8, 2020.0: 8, 2021.0: 8, 2022.0: 8, 2023.0: 8, 2024.0: 5}

Uzbekistan
Años con NaN: [2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2024.0]
Número de NaNs por año: {2010.0: 12, 2011.0: 12, 2012.0: 12, 2013.0: 12, 2014.0: 12, 2015.0: 12, 2016.0: 12, 2017.0: 12, 2018.0: 0, 2019.0: 0, 2020.0: 0, 2021.0: 0, 2022.0: 0, 2023.0: 0, 2024.0: 5}

United Arab Emirates
Años con NaN: [2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2024.0]
Número de NaNs por año: {2010.0: 12, 2011.0: 12, 2012.0: 12, 2013.0: 12, 2014.0: 12, 2015.0: 0, 2016.0: 0, 2017.0: 0, 2018.0: 0, 2019.0: 0, 2020.0: 0, 2021.0: 0, 2022.0: 0, 2023.0: 0, 2024.0: 4}

Poland
Años con NaN: [2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2023.0, 2024.0]
Núm

Se puede observar que se da el caso de países como pueden ser Uzbekistán o UAE los cuales no tienen datos importados en ciertos años:

![Datos faltantes de UAE y Uzbekistán](../data/images/UAE_uzbek.png)

Esto se debe a que nuestra fuente de datos no tiene dicha información disponible para esos años.

Otros países como Hungría, Polonia o Alabania tienen datos trimestrales, lo cual genera 8 NaNs por año:

![Datos faltantes de Hungría](../data/images/hungary.png)

Todo esto además se puede contrastar con las imágenes correspondientes de cada país guardadas en la carpeta **data/Tourism_jpgs/<XXXXXX>/<YYYY.png>** :

!['Hungria con datos trimestrales'](../data/Tourism_jpgs/Finland_Hungary_India_Malaysia\2010.png) 

En primer lugar vamos a eliminar los NaN de aquellos paises donde no tenemos data: 


        - Uzbekistan: eliminamos NaNs de 2010 a 2017 y los ultimos de 2024
        - UAE: eliminamos NaNs de 2010, 2011, 2012, 2013, 2014, ultimos meses 2024
        - Colombia: eliminamos NaNs de 2010,2011,2012,ultimos meses 2024
        - Vietnam: eliminamos NaNs de 2010 y último mes de 2024
        - Morocco: eliminamos NaNs de 2010 y último mes de 2024
        - Bulgaria: eliminamos NaNs de 2010


In [51]:
paises_nans =  ['United Arab Emirates', 'Colombia', 'Vietnam', 'Morocco', 'Bulgaria', 'Uzbekistan'] 
df = drop_nans(df,paises_nans)
print(df)

       Year  Month               Country  N_visitors
0      2010      1            Unnamed: 0           0
1      2010      1               Albania        <NA>
2      2010      1                 Egypt      998440
3      2010      1           Switzerland      535966
4      2010      2            Unnamed: 0           1
...     ...    ...                   ...         ...
10282  2023      8  United Arab Emirates     2132642
10283  2023      9  United Arab Emirates     2066071
10284  2023     10  United Arab Emirates     2342105
10285  2023     11  United Arab Emirates     2616600
10286  2023     12  United Arab Emirates     2828929

[10287 rows x 4 columns]


In [52]:
check_nans(df)

Total de NaNs: 273
Total desglosado:
Country
Hungary                     117
Poland                       61
Albania                      56
Switzerland                   8
United Kingdom                5
Egypt                         5
Malaysia                      4
Italy                         3
Denmark                       2
United States of America      2
India                         1
Indonesia                     1
Argentina                     1
Philippines                   1
Canada                        1
South Korea                   1
Sweden                        1
Belgium                       1
Tunisia                       1
Australia                     1
Name: N_visitors, dtype: int64


A continuación se realizará un tratamiento de aquellos países con data cada 3 meses.

Para ello se ha decidido realizar una interpolación lineal entre los datos, de esta forma a la hora de ejecutar las visualizaciones estas tendrán más coherencia.

Los países que se procesarán son:
    Hungary, Poland y Albania.

El programa "interpolar_nans":

    Interpola linealmente los NaNs en N_visitors solo para los países de la lista.
    
    Parámetros:
    df            : DataFrame con columnas 'Country', 'Year', 'Month', 'N_visitors'
    paises_interp : lista de países donde hacer interpolación
    
    Retorna:
    DataFrame con los NaNs interpolados en los países indicados



In [55]:
paises_interpolacion = ['Hungary', 'Poland', 'Albania']
df_interpolado = interpolar_nans(df, paises_interpolacion)

# Ver los primeros registros
print(df_interpolado.head(10))


   Year  Month      Country  N_visitors
0  2010      1   Unnamed: 0           0
1  2010      1      Albania        <NA>
2  2010      1        Egypt      998440
3  2010      1  Switzerland      535966
4  2010      2   Unnamed: 0           1
5  2010      2      Albania        <NA>
6  2010      2        Egypt     1018351
7  2010      2  Switzerland      590856
8  2010      3   Unnamed: 0           2
9  2010      3      Albania      201181


In [56]:
df_interpolado[df_interpolado['Country'] == 'Albania']

,Year,Month,Country,N_visitors
1,2010,1,Albania,<NA>
5,2010,2,Albania,<NA>
9,2010,3,Albania,201181
13,2010,4,Albania,266744
17,2010,5,Albania,332307
...,...,...,...,...
673,2024,1,Albania,469543
677,2024,2,Albania,462304
681,2024,3,Albania,537885
685,2024,4,Albania,764053


Alguno de los países interpolados mantienen NaNs:

In [59]:
check_nans(df_interpolado)

Total de NaNs: 45
Total desglosado:
Country
Switzerland                 8
United Kingdom              5
Egypt                       5
Malaysia                    4
Italy                       3
Hungary                     2
Poland                      2
Albania                     2
Denmark                     2
United States of America    2
India                       1
Indonesia                   1
Argentina                   1
Philippines                 1
Canada                      1
South Korea                 1
Sweden                      1
Belgium                     1
Tunisia                     1
Australia                   1
Name: N_visitors, dtype: int64


Los datos se han interpolado correctamente, sin embargo, seguimos teniendo 2 valores NaN en cada uno de estos paises debido a que no se pueden iterar al no tener un valor anterior al primero el cual comienza en el tercer mes.

Por ello se dropearán estos valores utilizando la función drop_nans:

In [61]:
df_2 = drop_nans(df_interpolado,paises_interpolacion)
check_nans(df_2)

Total de NaNs: 39
Total desglosado:
Country
Switzerland                 8
Egypt                       5
United Kingdom              5
Malaysia                    4
Italy                       3
United States of America    2
Denmark                     2
India                       1
Indonesia                   1
Australia                   1
Philippines                 1
South Korea                 1
Sweden                      1
Canada                      1
Tunisia                     1
Belgium                     1
Argentina                   1
Name: N_visitors, dtype: int64


Se ha reducido el total de NaNs a 39.


Estos se han revisado caso a caso, la mayoria son debido a falta de datos en ciertos anyos dentro del database, es decir, no existen o debido a fallas en la lectura por parte del OCR.


Se ha considerado dropearlos dado que no afectaran de forma significativa a nuestro analisis mas adelante

In [64]:
lista_paises = nans_pais.index.tolist() # Guardamos los indices para dropearlos en una lista
df_2 = drop_nans(df_2, lista_paises)
check_nans(df_2)

Total de NaNs: 0
Total desglosado:
Series([], Name: N_visitors, dtype: int64)


El siguiente paso será dropear las filas Unnamed: 0

In [65]:
indices = df_2[df_2['Country'] == 'Unnamed: 0'].index
df_2 = df_2.drop(indices)
df_2.head(30)

,Year,Month,Country,N_visitors
174,2010,1,Austria,2321719
176,2010,2,Austria,2441725
178,2010,3,Austria,2015283
180,2010,4,Austria,1205172
182,2010,5,Austria,1372359
184,2010,6,Austria,1576937
186,2010,7,Austria,2611357
188,2010,8,Austria,2708350
190,2010,9,Austria,1819056
192,2010,10,Austria,1261868


Check de duplicados:

In [66]:
duplicados_bool = df_2['N_visitors'].duplicated()
num_duplicados = duplicados_bool.sum()
print(f"Número de duplicados en N_visitors: {num_duplicados}")
df_duplicados = df_2[duplicados_bool]
print(df_duplicados)


Número de duplicados en N_visitors: 94
       Year  Month  Country  N_visitors
893    2020      5     Peru           0
896    2020      6     Peru           0
899    2020      7     Peru           0
902    2020      8     Peru           0
905    2020      9     Peru           0
...     ...    ...      ...         ...
10237  2024      1  Hungary     2856000
10238  2024      2  Hungary     2856000
10239  2024      3  Hungary     2856000
10240  2024      4  Hungary     2856000
10241  2024      5  Hungary     2856000

[94 rows x 4 columns]


A continuación guardo los datasets en csv para revisar los datos con mayor facilidad

In [67]:
df_2.to_csv("../data/df_2.csv", index=False) 
df_duplicados.to_csv("../data/df_duplicados.csv", index = False)

Tras hacer un check de los duplicados que aparecen, se observa una serie de duplicados generados en las últimas rows de Hungría y Polonia:

![Duplicados de Hungria y Polonia](../data/images/hungary_poland_dups.png)


Los duplicados de Hungria son debido a que el OCR ha guardado mal un dato en el csv de carga (tourism.csv), en vez de guardarlo en Hungria lo ha guardado en la India:

| ![](../data/images/india_hungary.png) | ![](../data/images/india_hungary2.png) |

Por ello realizaremos la correcion manual en el df_2 y eliminaremos los duplicados de forma manual (el archivo lo guardare como df_2_changed para que en caso de volver a ejecutar el programa de limpieza, no se pierdan los cambios manuales realizados).


Los ultimos valores de Hungría se interpolan a continuación nuevamente en df_3

In [70]:
df_3 = pd.read_csv("../data/df_2_changed.csv")
nans_pais = df_3.groupby('Country')['N_visitors'].apply(lambda x: x.isna().sum())
nans_hungary = nans_pais[nans_pais.index == "Hungary"]
print(nans_hungary)

Country
Hungary    4
Name: N_visitors, dtype: int64


In [71]:
# interpolamos
country = ['Hungary']
df_4 = interpolar_nans(df_3, country)
df_4[df_4['Country'] == 'Hungary'].tail()

,Year,Month,Country,N_visitors
5223,2024,1,Hungary,2731667
5224,2024,2,Hungary,2607333
5225,2024,3,Hungary,2483000
5226,2024,4,Hungary,2483000
5227,2024,5,Hungary,2483000


Ahora solo falta tratar los duplicados de Polonia

In [72]:
df_4[df_4['Country'] == 'Poland'].tail()

,Year,Month,Country,N_visitors
5052,2024,1,Poland,4238000
5053,2024,2,Poland,4238000
5054,2024,3,Poland,4238000
5055,2024,4,Poland,4238000
5056,2024,5,Poland,4238000


Como se puede ver en la imagen cargada por el OCR, no tenemos valores para 2024 de Poland:

![Poland 2024](../data/images/poland_2024.png)

Esta es la razon por la cual se duplican los valores al realizar la interpolacion, por lo cual dropearemos estos ultimos 4 resultados

In [73]:
rows_poland = df_4[(df_4['Country'] == 'Poland') & (df_4['Year'] == 2024)].index
df_4 = df_4.drop(rows_poland)
df_4[df_4['Country'] == 'Poland'].tail()



,Year,Month,Country,N_visitors
5047,2023,8,Poland,5731133
5048,2023,9,Poland,6101700
5049,2023,10,Poland,5480467
5050,2023,11,Poland,4859233
5051,2023,12,Poland,4238000


Una vez tratadom el csv, vamos a ordenarlo por Year, Month y Country para tener una visián más clara de este y los datos más ordenados.

In [74]:
df_4 = df_4.sort_values(by=['Country', 'Year', 'Month']).reset_index(drop=True)
df_4.head(30)

,Year,Month,Country,N_visitors
0,2010,3,Albania,201181
1,2010,4,Albania,266744
2,2010,5,Albania,332307
3,2010,6,Albania,397870
4,2010,7,Albania,756797
5,2010,8,Albania,1115723
6,2010,9,Albania,1474650
7,2010,10,Albania,1097645
8,2010,11,Albania,720641
9,2010,12,Albania,343636


El siguiente paso será realizar un cruce con el csv population_data.

Esto nos permitirá agregar nuevas columnas como la pobloación total anual de cada país así como el código del país.

Estos campos nos serán de ayuda a la hora generar gráficas para realizar por ejemplo comparativas entre turismo y población de un país o para insertar los códigos de país en leyendas para que sea más estético.

La función del cruce:

    Agrega la población y el código de país (Ref_country) a df_visitors por Country y Year, comparando nombres de países en minúsculas para no generar errores en el cruce
    La columna 'Population' se devuelve como Int64 para poder contener NaNs.
    La columna 'Ref_country' se devuelve como string.

In [81]:
# Leemos el fichero population_data
population_data = pd.read_csv("../data/population_data.csv")
df_5 = add_poblacion(df_4, population_data) #ejecutamos la función del merge
df_5.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8057 entries, 0 to 8056
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         8057 non-null   int64 
 1   Month        8057 non-null   int64 
 2   Country      8057 non-null   string
 3   N_visitors   8057 non-null   Int64 
 4   Population   7236 non-null   Int64 
 5   Ref_country  7236 non-null   string
dtypes: Int64(2), int64(2), string(2)
memory usage: 393.5 KB


Como podemos observar no se han traspasado todas las poblaciones dado que solo tenemos 7236 non-null vs 8057 non-null.


In [82]:
# Realizamos un check de los espacios vacios
nans_pais = df_5.groupby('Country')['Population'].apply(lambda x: x.isna().sum())
# Sacamos solo aquellos con nans mayores a cero
nans_pais = nans_pais[nans_pais > 0]
# Los ordenamos de mayor a menor
nans_pais = nans_pais.sort_values(ascending=False)
print(nans_pais)


Country
Turkey                      173
South Korea                 172
United States of America    171
Egypt                       168
Vietnam                     137
Name: Population, dtype: int64


Esto es debido a que los nombres de los paises difierene en ambos ficheros csv, por lo cual se procede a mapearlo y hacer un replace sobre population_data

In [83]:
# Diccionario de mapeo:
population_name_mapping = {
    "Turkey": "Turkiye",
    "South Korea": "Korea, Rep.",
    "United States of America": "United States",
    "Egypt": "Egypt, Arab Rep.",
    "Vietnam": "Viet Nam"
}

population_data['REF_AREA_LABEL'] = population_data['REF_AREA_LABEL'].replace({v: k for k, v in population_name_mapping.items()})

In [84]:
# Se vuelve a ejecutar la función add_poblacion
df_5 = add_poblacion(df_4, population_data)
df_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8057 entries, 0 to 8056
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         8057 non-null   int64 
 1   Month        8057 non-null   int64 
 2   Country      8057 non-null   string
 3   N_visitors   8057 non-null   Int64 
 4   Population   8057 non-null   Int64 
 5   Ref_country  8057 non-null   string
dtypes: Int64(2), int64(2), string(2)
memory usage: 393.5 KB


In [85]:
df_5.head()

,Year,Month,Country,N_visitors,Population,Ref_country
0,2010,3,Albania,201181,2913021,ALB
1,2010,4,Albania,266744,2913021,ALB
2,2010,5,Albania,332307,2913021,ALB
3,2010,6,Albania,397870,2913021,ALB
4,2010,7,Albania,756797,2913021,ALB


Como se puede observar ya no existen nulos.

El dataset se encuentra limpio y ordenado, por lo cual solo queda guardarlo en un nuevo csv:


In [86]:
df_5.to_csv("../data/tourism_clean.csv", index=False) # Guardamos el DataFrame final en un CSV para poder trabajar con el más adelante